In [1]:
%%capture
!pip install neptune-client psutil
!git clone https://github.com/Cho-D-YoungRae/URP_PD.git
%cd URP_PD

In [2]:
import dataset
import object_detection
from utils import *
import eval

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import torchvision.transforms.functional as TF
import torch.nn.functional as F

import os
import json
import numpy as np
import argparse
from tqdm.auto import tqdm
import time
from datetime import datetime

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using <{device}> device")

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Using <cuda> device


In [3]:
# ====== constants ======#
label_map = {'background': 0, 'person': 1}
rev_label_map = {v: k for k, v in label_map.items()} 

## setting

In [4]:
from torch.backends import cudnn
import random

cudnn.benchmark = True

# ====== Random Seed Initialization ====== #
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)

# torch.cuda.manual_seed(seed)
# torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False

parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.baselineID = 35

# ====== Dataset ====== #
args.ch_option = {'num_ch': 1,
                  'img_type': 'lwir',
                  'one_ch_option': 'mean'}
args.val_split = 0.1

# ====== Model ====== #
args.base_model = 'VGG16bnBase'
args.n_classes = len(label_map)
args.is_sds = True
args.usages_seg_feats = [True, True, False, False, False, False]
args.get_separate_loss = True


# ====== Optimizer & Training ====== #
args.optim = 'Adam'
args.lr = 5e-4
args.twice_b_lr = True
args.weight_decay = 5e-4

args.epochs = 150
args.train_batch_size = 32
args.test_batch_size = 64

args.decay_lr_at = [int(args.epochs/6)*4,
                    int(args.epochs/6)*5]
args.decay_lr_to = 0.1

## neptune init

In [5]:
import neptune.new as neptune

api_token = 
run = neptune.init(
    project='jodyr/urp',
    source_files=['*.py'],
    api_token=api_token,
    # run='PD-25',
    )

run["parameters"] = vars(args)


https://app.neptune.ai/jodyr/urp/e/PD-92
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


## train

In [6]:
from neptune.new.types import File

def train(train_loader, model, criterion, optimizer, run=None):
    """
    One epoch's training.

    :param train_loader: DataLoader for training data
    :param model: model
    :param criterion: MultiBox loss
    :param optimizer: optimizer
    :param epoch: epoch number
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    if criterion.get_separate_loss:
        sep_losses = {'conf_loss': AverageMeter(),
                      'loc_loss': AverageMeter(),
                      'total_seg_loss': AverageMeter()}

    print_freq = len(train_loader) // 4

    start = time.time()

    # Batches
    for i, (images, bboxes, category_ids, is_crowds, seg_labels) in enumerate(train_loader):
        data_time.update(time.time() - start)

        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        bboxes = [b.to(device) for b in bboxes]
        category_ids = [c.to(device) for c in category_ids]
        seg_labels = seg_labels.to(device)

        # Forward prop.
        predicted_locs, predicted_scores, pred_segs = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        if criterion.get_separate_loss:
            loss, separate_loss = criterion(
                predicted_locs, predicted_scores, bboxes,
                category_ids,pred_segs, seg_labels)  # scalar
            for loss_name, sep_loss in separate_loss.items():
                sep_losses[loss_name].update(sep_loss, images.size(0))

        else:
            loss = criterion(predicted_locs, predicted_scores, 
                             bboxes, category_ids, pred_segs, seg_labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()
        
        if i == 0 and run:
            ft_names = ['conv4_3', 'conv7', 'conv8_2', 'conv9_2', 'conv10_2', 'conv11_2']
            for ft_name, pred_seg in zip(ft_names, [pred_seg.detach() 
                                                    for pred_seg in pred_segs]):
                pred_seg = F.softmax(pred_seg, dim=1)
                pred_seg_grid = make_grid(pred_seg)
                pred_seg_grid = TF.to_pil_image(pred_seg_grid[1])
                run[f'train/pred_seg_{ft_name}'].log(File.as_image(pred_seg_grid))

            img_grid = make_grid(images)
            img_grid = TF.to_pil_image(img_grid)
            run['train/input'].log(File.as_image(img_grid))
            seg_gt_grid = make_grid(seg_labels)
            seg_gt_grid = TF.to_pil_image(seg_gt_grid)
            run['train/seg_gt'].log(File.as_image(seg_gt_grid))

        # Print status
        if i % print_freq == 0:
            print(f'[{i}/{len(train_loader)}]\t'
                  f'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  f'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  f'Loss {losses.val:.4f} ({losses.avg:.4f})\t')
    del predicted_locs, predicted_scores, images, bboxes, category_ids

    if run:
        run['train/loss'].log(losses.avg)
        if criterion.get_separate_loss:
            for loss_name, sep_loss in sep_losses.items():
                run[f'train/{loss_name}'].log(sep_loss.avg)

## validation

In [7]:
# def validation(val_loader, model, criterion):
#     model.eval()

#     num_batches = len(val_loader)
#     losses = AverageMeter()
#     with torch.no_grad():
#         for i, (images, bboxes, category_ids, _) in enumerate(val_loader):
#             images = images.to(device)
#             bboxes = [b.to(device) for b in bboxes]
#             category_ids = [l.to(device) for l in category_ids]

#             predicted_locs, predicted_scores = model(images)
#             loss = criterion(predicted_locs, predicted_scores, bboxes, category_ids).item()

#             losses.update(loss, images.size(0))

#     val_loss = losses.avg
#     return val_loss

## checkpoint

In [8]:
checkpoint = os.path.join('/content/drive/MyDrive/2021.summer_URP/PD/checkpoint',
                          str(args.baselineID)+'.pth.tar')
checkpoint = checkpoint if os.path.isfile(checkpoint) else None
print(f"checkpoint: {checkpoint}")

checkpoint: None


In [9]:
if checkpoint is None:
    start_epoch = 1
    lr = args.lr
    model = object_detection.SDSSSD300(n_classes=args.n_classes,
                                       base=args.base_model,
                                       ch_option=args.ch_option,
                                       usages_seg_feats=args.usages_seg_feats)
    if args.twice_b_lr:
        biases = list()
        not_biases = list()
        for param_name, param in model.named_parameters():
            if param.requires_grad:
                if param_name.endswith('.bias'):
                    biases.append(param)
                else:
                    not_biases.append(param)
        optimizer = getattr(torch.optim, args.optim)(params=[{'params': biases, 'lr': 2 * lr}, 
                                                            {'params': not_biases}],
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)
    else:
        optimizer = getattr(torch.optim, args.optim)(params=model.parameters(),
                                                     lr=lr,
                                                     weight_decay=args.weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']


model = model.to(device)
criterion = object_detection.SDSMultiBoxLoss(
    priors_cxcy=model.priors_cxcy, 
    usages_seg_feats=model.usages_seg_feats,
    get_separate_loss=args.get_separate_loss).to(device)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth




Loaded base model.



/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


## dataset init

In [10]:
workers = 4
train_dataset = dataset.KaistPDDataset(ch_option=args.ch_option,
                                       is_sds=args.is_sds)
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                           batch_size=args.train_batch_size, 
                                           shuffle=True,
                                           collate_fn=dataset.sds_collate_fn,
                                           num_workers=workers,
                                           pin_memory=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## experiment

In [11]:
checkpoint_dir = '/content/drive/MyDrive/2021.summer_URP/PD/checkpoint'
checkpoint_path = os.path.join(checkpoint_dir,
                               str(args.baselineID)+'.pth.tar')

In [12]:
epochs = args.epochs
decay_lr_at = args.decay_lr_at
save_freq = 5
eval_freq = 10

# Epochs
for epoch in range(start_epoch, epochs+1):
    print(f"# ====== Epoch {epoch} ====== # {datetime.now()}")
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, args.decay_lr_to)

    # One epoch's training
    train(train_loader=train_loader, model=model,
          criterion=criterion, optimizer=optimizer, run=run)
    if epoch % save_freq == 0:
        save_checkpoint(epoch, model, optimizer, checkpoint_path)

    if epoch % eval_freq == 0:
        eval.evaluate(model, ch_option=args.ch_option)

# ====== Epoch 1 ====== # 2021-08-06 02:16:10.806280


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[0/273]	Batch Time 102.461 (102.461)	Data Time 96.718 (96.718)	Loss 13.3402 (13.3402)	
[68/273]	Batch Time 28.164 (10.206)	Data Time 27.611 (9.587)	Loss 4.7815 (5.6910)	
[136/273]	Batch Time 26.288 (9.505)	Data Time 25.748 (8.924)	Loss 4.7089 (5.1709)	
[204/273]	Batch Time 16.950 (9.222)	Data Time 16.415 (8.654)	Loss 4.6301 (4.9815)	
[272/273]	Batch Time 9.961 (9.044)	Data Time 5.364 (8.468)	Loss 4.4963 (4.8766)	
# ====== Epoch 2 ====== # 2021-08-06 02:57:19.958687
[0/273]	Batch Time 4.770 (4.770)	Data Time 4.089 (4.089)	Loss 4.5350 (4.5350)	
[68/273]	Batch Time 0.801 (1.057)	Data Time 0.172 (0.433)	Loss 4.1901 (4.3374)	
[136/273]	Batch Time 1.854 (1.052)	Data Time 1.251 (0.430)	Loss 3.5943 (4.1065)	
[204/273]	Batch Time 2.421 (1.038)	Data Time 1.797 (0.414)	Loss 3.3714 (3.9206)	
[272/273]	Batch Time 0.472 (1.026)	Data Time 0.000 (0.404)	Loss 3.2544 (3.8071)	
# ====== Epoch 3 ====== # 2021-08-06 03:02:00.025286
[0/273]	Batch Time 5.088 (5.088)	Data Time 4.447 (4.447)	Loss 3.3130 (3.313

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.35s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 33.09%
Recall: 0.7959039548022598
# ====== Epoch 11 ====== # 2021-08-06 04:01:00.258323


...


Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.29s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 27.29%
Recall: 0.808045165843331
# ====== Epoch 141 ====== # 2021-08-06 14:19:45.616584


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0/273]	Batch Time 4.371 (4.371)	Data Time 3.641 (3.641)	Loss 2.2928 (2.2928)	
[68/273]	Batch Time 0.646 (1.054)	Data Time 0.001 (0.429)	Loss 2.2244 (2.0817)	
[136/273]	Batch Time 2.117 (1.042)	Data Time 1.500 (0.417)	Loss 2.6121 (2.0983)	
[204/273]	Batch Time 1.720 (1.031)	Data Time 1.122 (0.406)	Loss 2.0262 (2.0983)	
[272/273]	Batch Time 0.469 (1.011)	Data Time 0.000 (0.388)	Loss 2.1815 (2.0944)	
# ====== Epoch 142 ====== # 2021-08-06 14:24:21.841185
[0/273]	Batch Time 4.707 (4.707)	Data Time 4.085 (4.085)	Loss 2.0232 (2.0232)	
[68/273]	Batch Time 2.422 (1.036)	Data Time 1.790 (0.410)	Loss 2.1343 (2.0681)	
[136/273]	Batch Time 1.489 (1.013)	Data Time 0.871 (0.387)	Loss 2.2942 (2.0700)	
[204/273]	Batch Time 0.630 (1.008)	Data Time 0.001 (0.382)	Loss 1.8208 (2.0755)	
[272/273]	Batch Time 0.469 (1.003)	Data Time 0.000 (0.379)	Loss 2.2191 (2.0788)	
# ====== Epoch 143 ====== # 2021-08-06 14:28:55.791146
[0/273]	Batch Time 4.832 (4.832)	Data Time 4.194 (4.194)	Loss 2.2324 (2.2324)	
[68/273

/content/URP_PD/object_detection.py:395: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_boxes.append(class_decoded_locs[1 - suppress])
/content/URP_PD/object_detection.py:397: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:30.)
  image_scores.append(class_scores[1 - suppress])



Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.28s).
Accumulating evaluation results...
DONE (t=0.18s).
 Average Miss Rate  (MR) @ Reasonable         [ IoU=0.50      | height=[55:10000000000] | visibility=[none+partial_occ] ] = 27.34%
Recall: 0.8070671378091873
